In [1]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier

import warnings
warnings.simplefilter('ignore')

In [2]:
got_df = pd.read_csv("character-predictions.csv")

In [3]:
# Data preparation
got_df.age[got_df.age < 0] = 0
got_df.age = got_df.age.fillna(int(got_df.age.mean()))
got_df = got_df.fillna(0)

In [4]:
X = got_df[['male', 'book1', 'book2', 'book3', 'book4', 'book5', 'isAliveMother', 'isAliveFather', 'isAliveHeir', 'isAliveSpouse', 'isMarried', 'isNoble', 'age', 'numDeadRelations', 'boolDeadRelations',
       'isPopular', 'popularity']]
y = got_df['actual']
X.head()

,male,book1,book2,book3,book4,book5,isAliveMother,isAliveFather,isAliveHeir,isAliveSpouse,isMarried,isNoble,age,numDeadRelations,boolDeadRelations,isPopular,popularity
0,1,0,0,0,0,0,1.0,0.0,0.0,0.0,0,0,36.0,11,1,1,0.605351
1,1,1,1,1,1,1,0.0,0.0,0.0,1.0,1,1,97.0,1,1,1,0.896321
2,1,0,0,0,1,0,0.0,0.0,0.0,0.0,0,1,36.0,0,0,0,0.267559
3,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1,1,23.0,0,0,0,0.183946
4,0,0,0,0,1,0,0.0,0.0,0.0,1.0,1,1,29.0,0,0,0,0.043478


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)

In [6]:
clf_lr = LogisticRegression(class_weight='balanced')
clf_dt = DecisionTreeClassifier(min_samples_leaf=3, min_samples_split=9, random_state=500)
clf_svm = SVC(probability=True, class_weight='balanced', random_state=500)

In [7]:
# Build and fit an averaging classifier
clf_avg = VotingClassifier(
    estimators=[
        ('svm', clf_svm), 
        ('lr', clf_lr), 
        ('dt', clf_dt)],
    voting='soft'
    # Is posible give more weights to some models than others
    #weights=[1, 2, 1]
)

In [8]:
clf_avg.fit(X_train, y_train)

VotingClassifier(estimators=[('svm', SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=500,
  shrinking=True, tol=0.001, verbose=False)), ('lr', LogisticRegression(C=1.0, class...       min_weight_fraction_leaf=0.0, presort=False, random_state=500,
            splitter='best'))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None)

In [9]:
# Evaluate model performance
acc_avg = accuracy_score(y_test,  clf_avg.predict(X_test))
print('Accuracy: {:.2f}'.format(acc_avg))

Accuracy: 0.78
